## SI 670 Applied Machine Learning, Week 6:  Naive Bayes, Boosting Algorithms, Pipeline. (Due 10/12 11:59pm)

For this assignment, you will be revisiting concepts including Naive Bayes, boosting algorithms and pipelines.

* This homework is worth 100 points in total. Correct answers and code receive full credit, but partial credit will be awarded if you have the right idea even if your final answers aren't quite right.

* Submit your completed notebook file to the Canvas site - **IMPORTANT**: please name your submitted file `si670f22-hw6-youruniqname.ipynb`

* Any file submitted after the deadline will be marked as late. Please consult the syllabus regarding late submission policies. You can submit the homework as many time as you want, but only your latest submission will be graded.

* As a reminder, the notebook code you submit must be your own work. Feel free to discuss general approaches to the homework with classmates. If you end up forming more of a team discussion on multiple questions, please include the names of the people you worked with at the top of your notebook file.


### Collaborators, if any:

### Question 1 (20 points)

Please write the answers as well as your derivation process of the following questions. You can use either LaTeX or python code to represent your answer. For example, if you want to present <$x_1^2$>, in the LaTeX format you should write <(dollar sign) x_1^2 (dollar sign)>; in the python code format you should write <\`x_1\*\*2\`>. See [here](https://csrgxtu.github.io/2015/03/20/Writing-Mathematic-Fomulars-in-Markdown/) for how to represent more mathmatical symbols in LaTeX format.

**Calculate the unnormalized posterior probability of a naive Bayes classifier**

Suppose you have a dataset with 2 features $X_1, X_2$ and a binary label $Y$. $X_1$ and $Y$ takes value either 0 or 1. $X_2$ takes one out of the three possible values $0, 1$, or $2$.

Based on the dataset, you know 

$p(Y=0) = 0.05$, 

$p(X_1=0 | Y=0) = 0.6$, $p(X_1=0 | Y=1) = 0.3$, 

$p(X_2=0 | Y=0) = 0.9$, $p(X_2=1 | Y=0) = 0.05$, $p(X_2=0 | Y=1) = 0.1$, $p(X_2=1 | Y=1) = 0.3$. 

Please calculate the unnormalized posterior probability of a naive Bayes classifier: 
$$\hat{p}(Y=1 | X) = p(X | Y=1) p(Y=1)$$ and $$\hat{p}(Y=0 | X) = p(X | Y=0) p(Y=0)$$ of the following data points.

#### (a) (10 points) $X_1 = 1, X_2 = 0$



#### (b) (10 points) $X_1 = 0, X_2 = 2$



### Answer 1

(a) 

$$\hat{p}(Y=1 | X) = p(X | Y=1) p(Y=1)$$ = $$p(X_1=1| Y=1)p(X_2=0 | Y=1) (1-p(Y=0))$$ = $$(1-0.3)*(0.1)*(0.95)$$ = $$0.7*0.1*0.95$$ = 0.0665

$$\hat{p}(Y=0 | X) = p(X | Y=0) p(Y=0)$$ = $$p(X_1=1| Y=0)p(X_2=0 | Y=0) p(Y=0)$$ = $$(1-0.6)(0.9)(0.05)$$ = 0.018



(b)
$$\hat{p}(Y=1 | X) = p(X | Y=1) p(Y=1)$$ = $$p(X_1=0| Y=1)p(X_2=2 | Y=1) (1-p(Y=0))$$ = $$(0.3)*(1-0.1-0.3)*(1-0.05)$$ = 0.171

$$\hat{p}(Y=0 | X) = p(X | Y=0) p(Y=0)$$ = $$p(X_1=0| Y=0)p(X_2=2 | Y=0) p(Y=0)$$ = $$0.6*(1-0.9-0.05)*(0.05)$$ = 0.0015

### Question 2 (40 points)

For this question, you will be implementing a gradient boosted decision tree using the `GradientBoostedClassifier` from `sklearn`. Your goal is to find the best set of hyperparameters for this classifier using the tools you have seen previously in this class.

#### (a) (10 points) 
Consider the parameters `learning_rate` and `n_estimators` (which represents the number of small trees or weak learners). The documentation for `GradientBoostingClassifier` mentions that "There is a trade-off between learning_rate and n_estimators". Why do you think this is the case?

* Your answer for 2(a) here:
* n_estimators sets # of small decision trees to use (weak learners) in the ensemble, which can cause overfit, the learning rate controls emphasis on fixing errors from previous iteration, which can shrink the contribution of each tree. Control overfit. so it is a trade-off between these two. Usually the n_estimators is adjusted first, and tune with the learning rate together.  

#### (b) (20 points) 

Now for the implementation part. First, use `MinMaxScaler` to scale the breast cancer data (be careful about data leakage). Then, use `GridSearchCV` (similar to how you have used it in previous homeworks) in order to find the best `learning_rate`, `n_estimators` and `max_depth` parameters for a gradient boosted decision tree implemented by `GradientBoostingClassifier`.

Please search `learning_rate` from `[0.1, 1, 10]`, `n_estimators` from `[1, 5, 10, 100]`, and `max_depth` from `[3, 5]`.

Please return the best hyperparameters on cross-validation, as well as the training and test scores associated with these hyperparameters.

In [9]:
from sklearn import model_selection
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

def run_gridsearch_gboost():
    # load data
    (X_cancer, y_cancer) = load_breast_cancer(return_X_y = True)
    X_train, X_test, y_train, y_test = train_test_split(X_cancer, y_cancer, random_state = 0)

    # scale
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # tune with GridSearchCV
    parameters = {'learning_rate':[0.1,1,10], 'n_estimators':[1, 5, 10, 100], 'max_depth':[3,5]}
    clf = GridSearchCV(GradientBoostingClassifier(), parameters)
    clf.fit(X_train_scaled, y_train)
    
    # best parameters
    best_learning_rate = clf.best_params_['learning_rate']
    best_n_estimators = clf.best_params_['n_estimators']
    best_max_depth = clf.best_params_['max_depth']
    
    # test
    test_score = clf.score(X_test_scaled, y_test)
    # GridSearchCV do performed cross-validation

    return best_learning_rate, best_n_estimators, best_max_depth, test_score

run_gridsearch_gboost()

(0.1, 100, 3, 0.972027972027972)

#### (c) (10 points)

So far in lab and homework problems, we have been using scalers to scale the training and test data before applying `GridSearchCV` to find the best model. Does this approach systematically bias the test score? Why, then, should you not use this approach in practice? *(Hint: think about validation)*

* Your answer for 2(c) here:
* Yes, this approach (scale before applying GridSearchCV) will systematically bias the test score. Because the validation data in cross validation part will use the scaled and data leakaged data and info.

* Instead, we can use pipelines. We can assemble the scaler and models in a pipeline then use it as the estimator for GridSearchCV. Pipeline will automatically fit and transform your training data in each fold, and transform on validation set using fitted scaler.

### Question 3 (40 points)

We will solve the issue raised in 2(c) here using a pipeline. While you are answer this question, think about why a pipeline is helpful in addressing the issue from 2(c) (You do not need to write down your thoughts for this question).

Build a pipeline that first applies the MinMaxScaler to the data and then grid searches the hyper-parameter `C` of the `LinearSVC` classifier. Note the whole process should be wrapped in a single pipeline.

Return a tuple `(pipe, test_score)`, where `pipe` is the pipeline object you build and `test_score` is the accuracy score you get from your final model on `(X_test, y_test)`.

The grid search range of the parameter is given in `param_grid`.

*Hint1: The `GridSearchCV` itself can be viewed as a classifier or a regressor because it implements `.fit` and `.score` functions.*

*Hint2: There are two ways to combine `GridSearchCV` and `Pipeline`: you can either grid search a pipeline with the scaler and the classifier; or you can pipeline the scaler and the "grid-search classifier". This question requires you to implement the latter one. Think about which way is more computationally efficient?*

In [10]:
def answer_three():
    from sklearn.pipeline import Pipeline
    from sklearn.svm import LinearSVC
    from sklearn.datasets import load_breast_cancer
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import train_test_split

    # get data, split data
    (X_cancer, y_cancer) = load_breast_cancer(return_X_y = True)
    X_train, X_test, y_train, y_test = train_test_split(X_cancer, y_cancer, random_state = 0)

    # parameter
    param_grid = {"clf__C": [0.1, 1, 10, 100]}
    
    # build pipeline
    components = [('scaler', MinMaxScaler()), ('clf', LinearSVC())]
    pipe = Pipeline(components)
    
    # grid search a pipeline (with the scaler and the classifier)
    grid = GridSearchCV(pipe, param_grid) 
    
    # GridSearchCV itself can be viewed as a classifier, can implement .fit and .score functions
    grid.fit(X_train, y_train)
    test_score = grid.score(X_test, y_test)

    return pipe, grid, test_score

answer_three()

C:\Users\Jiaqi\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Jiaqi\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Jiaqi\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Jiaqi\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Jiaqi\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Jiaqi\AppData\Roaming\Python\Python39\sit

(Pipeline(steps=[('scaler', MinMaxScaler()), ('clf', LinearSVC())]),
 GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                        ('clf', LinearSVC())]),
              param_grid={'clf__C': [0.1, 1, 10, 100]}),
 0.958041958041958)

* I return both the pipe and grid, because the changing announcement did not say return which one.

* Think about which way is more computationally efficient?

1. grid search a pipeline with the scaler and the classifier; 
2. pipeline the scaler and the "grid-search classifier"

* The 2nd method will call the Scalar only once when fit(), so it might be more computationally efficient, but it leads to data leakage (tune the hyperparameters on the data already preprocessed by Scaler).